In [ ]:
# pip install xgboost

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:
from sklearn.preprocessing import StandardScaler, PolynomialFeatures

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
!pip install imbalanced-learn

In [ ]:
from imblearn.over_sampling import ADASYN

In [ ]:
!pip install tslearn

In [ ]:
from tslearn.clustering import TimeSeriesKMeans

In [ ]:
# Load the data
data = pd.read_csv('EDA4_nan820.csv')
X = data.drop(['trending', 'Time'], axis=1)
y = data['trending']

# Split the data at line 820
split_index = 820
X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Polynomial Feature Transformation
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Resample the training data using ADASYN
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_poly, y_train)

# Hyperparameter tuning using GridSearchCV
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search = GridSearchCV(RandomForestClassifier(random_state=42), param_grid, cv=3)
grid_search.fit(X_train_adasyn, y_train_adasyn)
best_clf = grid_search.best_estimator_

# Predict using the best model
y_pred_proba = best_clf.predict_proba(X_test_poly)[:, 1]

# Define thresholds
thresholds = np.linspace(0, 1, 100)
y_pred_proba

In [ ]:
# Predict on the training set using the best model
y_train_pred_proba = best_clf.predict_proba(X_train_poly)[:, 1]

# Convert probabilities to binary predictions using a threshold of 0.5
y_train_pred = (y_train_pred_proba > 0.5).astype(int)

# Calculate metrics for the training set
accuracy_train = accuracy_score(y_train, y_train_pred)
precision_train = precision_score(y_train, y_train_pred)
recall_train = recall_score(y_train, y_train_pred)
f1_train = f1_score(y_train, y_train_pred)

accuracy_train, precision_train, recall_train, f1_train

In [ ]:
# Load the data
data = pd.read_csv('EDA4_nan820.csv')
X = data.drop(['trending', 'Time'], axis=1)
y = data['trending']

# Split the data at line 820
split_index = 820
X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]

# Preprocess the test data
X_test_scaled = scaler.transform(X_test)
X_test_poly = poly.transform(X_test_scaled)

# Predict using the trained model
y_test_pred_proba = best_clf.predict_proba(X_test_poly)[:, 1]
y_test_pred = (y_test_pred_proba > 0.5).astype(int)

y_test_pred

In [ ]:
# # Combining the training and test data
# combined_data = data.copy()
# combined_data['Predicted_Trending'] = np.concatenate([y_train, y_test_pred])

# # Saving the combined data to a CSV file
# combined_data.to_csv('newAUG.csv', index=False)

In [ ]:
# Load the corrected data
corrected_data = pd.read_csv('Corrected_Predictions_nan1190.csv')
X_corrected = corrected_data.drop(['Corrected_Predictions', 'trending', 'Time'], axis=1)
y_corrected = corrected_data['Corrected_Predictions']

# Split the data at line 1190
split_index_corrected = 1190
X_train_corrected = X_corrected.iloc[:split_index_corrected]
y_train_corrected = y_corrected.iloc[:split_index_corrected]

# Scale the features
scaler_corrected = StandardScaler()
X_train_scaled_corrected = scaler_corrected.fit_transform(X_train_corrected)

# Apply Polynomial Feature Transformation
poly_corrected = PolynomialFeatures(degree=2)
X_train_poly_corrected = poly_corrected.fit_transform(X_train_scaled_corrected)

# Resample the training data using ADASYN
adasyn_corrected = ADASYN(random_state=42)
X_train_adasyn_corrected, y_train_adasyn_corrected = adasyn_corrected.fit_resample(X_train_poly_corrected, y_train_corrected)

# Hyperparameter tuning using GridSearchCV
param_grid_corrected = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search_corrected = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_corrected, cv=3)
grid_search_corrected.fit(X_train_adasyn_corrected, y_train_adasyn_corrected)
best_clf_corrected = grid_search_corrected.best_estimator_

In [ ]:
# Predict using the retrained model on the training data
y_pred_corrected = best_clf_corrected.predict(X_train_poly_corrected)

# Calculate accuracy, precision, recall, and f1 score
accuracy_corrected = accuracy_score(y_train_corrected, y_pred_corrected)
precision_corrected = precision_score(y_train_corrected, y_pred_corrected)
recall_corrected = recall_score(y_train_corrected, y_pred_corrected)
f1_corrected = f1_score(y_train_corrected, y_pred_corrected)

accuracy_corrected, precision_corrected, recall_corrected, f1_corrected

In [ ]:
# Predict using the retrained model on the test data
y_pred_test = best_clf_corrected.predict(X_test_poly)

# Since the test set target values are NaN, we can't calculate accuracy, precision, recall, and f1 score directly.
# Instead, we'll return the predictions for further analysis.
y_pred_test

In [ ]:
# Load the corrected data
data_corrected = pd.read_csv('Corrected_Predictions_nan1190.csv')

# Add the training predictions to the 'Model_Predictions' column
data_corrected.loc[:split_index-1, 'Model_Predictions'] = y_train_corrected

# Add the test predictions to the 'Model_Predictions' column
data_corrected.loc[split_index:, 'Model_Predictions'] = y_pred_test

# Save the combined dataset with predictions to a new CSV file
data_corrected.to_csv('analyse.csv', index=False)

'analyse.csv saved successfully.'

In [ ]:
import joblib

# Save the trained model to a file
model_filename = 'Retrained_trending_modelAUG30.pkl'
joblib.dump(best_clf, model_filename)

f'Model saved as {model_filename}'

In [ ]:
# Modified hyperparameter grid
param_grid_modified = {
    'n_estimators': [100, 150, 200],
    'max_depth': [None, 20, 30, 40, 50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Hyperparameter tuning using GridSearchCV with the modified parameter grid
grid_search_modified = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_modified, cv=3)
grid_search_modified.fit(X_train_adasyn, y_train_adasyn)
best_clf_modified = grid_search_modified.best_estimator_

# Display the best parameters from the grid search
grid_search_modified.best_params_

In [ ]:
import xgboost as xgb
from sklearn.ensemble import VotingClassifier

# Load the data
data = pd.read_csv('EDA4_nan820.csv')
X = data.drop(['trending', 'Time'], axis=1)
y = data['trending']

# Split the data at line 820
split_index = 820
X_train = X.iloc[:split_index]
X_test = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test = y.iloc[split_index:]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Apply Polynomial Feature Transformation
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)
X_test_poly = poly.transform(X_test_scaled)

# Resample the training data using ADASYN
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_poly, y_train)

# Train RandomForest
param_grid_rf = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}
grid_search_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_grid_rf, cv=3)
grid_search_rf.fit(X_train_adasyn, y_train_adasyn)
best_rf = grid_search_rf.best_estimator_

# Train XGBoost with custom loss function for trend length
def custom_loss(y_true, y_pred):
    # Calculate the difference between consecutive predictions
    diff = np.diff(y_pred)
    # Penalize short trends
    penalty = np.sum(np.abs(diff))
    # Standard log loss
    log_loss = -np.sum(y_true * np.log(y_pred) + (1 - y_true) * np.log(1 - y_pred))
    return log_loss + penalty

param_grid_xgb = {
    'n_estimators': [50, 100, 150],
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.05, 0.1],
    'alpha': [0, 0.1, 1],
    'lambda': [0, 0.1, 1]
}
grid_search_xgb = GridSearchCV(xgb.XGBClassifier(objective='binary:logistic', eval_metric='logloss', random_state=42), param_grid_xgb, cv=3)
grid_search_xgb.fit(X_train_adasyn, y_train_adasyn)
best_xgb = grid_search_xgb.best_estimator_

# Ensemble the models using VotingClassifier
ensemble_model = VotingClassifier(estimators=[('rf', best_rf), ('xgb', best_xgb)], voting='soft')
ensemble_model.fit(X_train_adasyn, y_train_adasyn)

# Predictions from the ensemble model
y_train_pred_ensemble = ensemble_model.predict(X_train_poly)

In [ ]:
import matplotlib.pyplot as plt

# Extracting the actual and predicted values
actual_values = y_train
predicted_values = y_train_pred_ensemble

# Plotting the actual vs predicted values
plt.figure(figsize=(15, 7))
plt.plot(actual_values.values, label='Actual Trending Values', color='blue')
plt.plot(predicted_values, label='Predicted Trending Values', color='red', linestyle='dashed')
plt.title('Actual vs Predicted Trending Values')
plt.xlabel('Index')
plt.ylabel('Trending Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Preprocess the test data
X_test_scaled = scaler.transform(X_test)
X_test_poly = poly.transform(X_test_scaled)

# Predict using the ensemble model
y_test_pred_ensemble = ensemble_model.predict(X_test_poly)

In [ ]:
# Plotting the ensemble model predictions for the test set
plt.figure(figsize=(15, 7))
plt.plot(y_test.index, y_test_pred_ensemble, label='Predicted Trending Values', color='blue')
plt.title('Ensemble Model Predictions for Test Set')
plt.xlabel('Index')
plt.ylabel('Trending')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Concatenate the training and test data
full_data = pd.concat([X_train, X_test])
full_data['Actual_Trending'] = pd.concat([y_train, y_test])
full_data['Ensemble_Predictions'] = np.concatenate([y_train_pred_ensemble, y_test_pred_ensemble])

# Save the dataframe to a CSV file
full_data.to_csv('ensamble_analyse.csv', index=False)

'ensamble_analyse.csv saved successfully.'

In [ ]:
import joblib

# Save the ensemble model to a file
joblib.dump(ensemble_model, 'hybrid_ensamble.pkl')

In [ ]:
from joblib import load

ensemble_model = load('hybrid_ensamble.pkl')

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import matplotlib.pyplot as plt

# Load the data
data = pd.read_csv('Corrected_Predictions_nan1190.csv')
X = data.drop(['Corrected_Predictions', 'Time', 'trending'], axis=1)
y = data['Corrected_Predictions']

# Split the data at line 1190
split_index = 1190
X_train = X.iloc[:split_index]
y_train = y.iloc[:split_index]

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Apply Polynomial Feature Transformation
poly = PolynomialFeatures(degree=2)
X_train_poly = poly.fit_transform(X_train_scaled)

# Resample the training data using ADASYN
adasyn = ADASYN(random_state=42)
X_train_adasyn, y_train_adasyn = adasyn.fit_resample(X_train_poly, y_train)

# Train RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
rf_clf.fit(X_train_adasyn, y_train_adasyn)

# Train XGBoost
def custom_loss(y_pred, dtrain):
    y_true = dtrain.get_label()
    grad = np.where(y_true == 1, -2*(1-y_pred), 2*y_pred)
    hess = np.where(y_true == 1, 2*y_pred, 2*(1-y_pred))
    return grad, hess

dtrain = xgb.DMatrix(X_train_adasyn, label=y_train_adasyn)
params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'max_depth': 6,
    'eta': 0.1,
    'alpha': 1,
    'lambda': 1
}
bst = xgb.train(params, dtrain, num_boost_round=1000, obj=custom_loss)

# Ensemble predictions
class EnsembleModel:
    def __init__(self, model1, model2):
        self.model1 = model1
        self.model2 = model2

    def predict(self, X):
        # Predict trend length with XGBoost
        trend_length_pred = self.model2.predict(xgb.DMatrix(X))
        # Use trend length prediction to adjust RandomForest predictions
        rf_pred = self.model1.predict(X)
        adjusted_pred = np.where(trend_length_pred > 0.5, rf_pred, 0)
        return adjusted_pred

ensemble_model = EnsembleModel(rf_clf, bst)
y_train_pred_ensemble = ensemble_model.predict(X_train_poly)

# Calculate metrics
accuracy = accuracy_score(y_train, y_train_pred_ensemble)
precision = precision_score(y_train, y_train_pred_ensemble)
recall = recall_score(y_train, y_train_pred_ensemble)
f1 = f1_score(y_train, y_train_pred_ensemble)

accuracy, precision, recall, f1

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(y_train.values, label='Actual Values', color='blue')
plt.plot(y_train_pred_ensemble, label='Predicted Values', color='red', linestyle='--')
plt.title('Actual vs Predicted Values')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Check the number of non-NaN and NaN values in y_test
non_nan_count = y_test.dropna().shape[0]
nan_count = y_test.isna().sum()
non_nan_count, nan_count

In [ ]:
# Check the shape and first few rows of X_test
X_test.shape, X_test.head()

In [ ]:
# Load the data from 'Corrected_Predictions_nan1190.csv'
data_corrected = pd.read_csv('Corrected_Predictions_nan1190.csv')
X_corrected = data_corrected.drop(['Corrected_Predictions', 'Time', 'trending'], axis=1)
y_corrected = data_corrected['Corrected_Predictions']
# Extract rows with NaN values in y_corrected for prediction
X_missing_corrected = X_corrected[y_corrected.isna()]
# Predict values for the missing data
y_missing_pred_corrected = ensemble_model.predict(poly.transform(scaler.transform(X_missing_corrected)))
y_missing_pred_corrected

In [ ]:
plt.figure(figsize=(14, 6))
plt.plot(y_missing_pred_corrected, label='Predicted Values', color='red')
plt.title('Predicted Values for Missing Data')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Add the predicted values to the 'Corrected_Predictions' column for the rows with NaN values
data_corrected.loc[y_corrected.isna(), 'Corrected_Predictions'] = y_missing_pred_corrected
# Save the updated dataset to 'hybrid2.csv'
data_corrected.to_csv('hybrid2.csv', index=False)

In [ ]:
# Reload necessary libraries and data
import pandas as pd
import matplotlib.pyplot as plt

# Load the new CSV data into a DataFrame
data_aug = pd.read_csv('hybrid2.csv')

# Plot the data
plt.figure(figsize=(15, 7))

# Plot the Open, High, Low, Close columns
data_aug[['Open', 'High', 'Low', 'Close']].plot(ax=plt.gca(), title='OHLC Chart with Predicted_Trending')

# Create a secondary y-axis to plot the 'Predicted_Trending' column
data_aug['Corrected_Predictions'].plot(secondary_y=True, style='g', ax=plt.gca(), alpha=0.7, label='Predicted_Trending')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
# Remove the first 820 rows from the dataframe
data_aug_trimmed = data_aug.iloc[820:].reset_index(drop=True)

# Plot the trimmed data
plt.figure(figsize=(15, 7))

# Plot the Open, High, Low, Close columns
data_aug_trimmed[['Open', 'High', 'Low', 'Close']].plot(ax=plt.gca(), title='OHLC Chart with Predicted_Trending (After removing first 820 rows)')

# Create a secondary y-axis to plot the 'Predicted_Trending' column
data_aug_trimmed['Corrected_Predictions'].plot(secondary_y=True, style='g', ax=plt.gca(), alpha=0.7, label='Predicted_Trending')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
# Zoom in to the first 100 data points after removing the initial 820 rows
data_aug_zoomed = data_aug_trimmed.iloc[:300]

# Plot the zoomed-in data
plt.figure(figsize=(15, 7))

# Plot the Open, High, Low, Close columns
data_aug_zoomed[['Open', 'High', 'Low', 'Close']].plot(ax=plt.gca(), title='OHLC Chart (Zoomed to first 100 data points) with Predicted_Trending')

# Create a secondary y-axis to plot the 'Predicted_Trending' column
data_aug_zoomed['Corrected_Predictions'].plot(secondary_y=True, style='g', ax=plt.gca(), alpha=0.7, label='Predicted_Trending')
plt.legend(loc='best')
plt.grid(True)
plt.show()

In [ ]:
import joblib
# Save the ensemble model to a file
joblib.dump(ensemble_model, 'hybrid_second_training.pkl')